### Install dependencies

In [1]:
# !pip install qiskit

In [2]:
# !pip install qiskit_algorithms

In [3]:
# !pip install qiskit_optimization

In [4]:
# ! pip install qiskit-aer

In [5]:
# !pip install pylatexenc

In [6]:
# !pip install gurobipy

In [7]:
# !pip install deap

In [8]:
# ! python -m pip install --upgrade pip

In [9]:
# !pip install dwave-ocean-sdk

In [10]:
# !pip install dimod

In [11]:
# !pip install qiskit-aqua

In [12]:
# !pip install pennylane

### Import dependencies

In [14]:
# useful additional packages

from qiskit.circuit.library import Diagonal
from qiskit import QuantumCircuit


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx
import os


from qiskit_optimization.applications import Maxcut
from qiskit_optimization.algorithms import MinimumEigenOptimizer


from qiskit.primitives import Sampler

from qiskit.result import QuasiDistribution


from scipy.optimize import minimize
import time

In [15]:
# UPDATED MODULES

from qiskit_algorithms.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA

from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator

from qiskit_algorithms.utils import algorithm_globals


In [16]:
import gurobipy as gp
from gurobipy import GRB

In [17]:
from deap import base, creator, tools, algorithms

In [18]:
import random

## Experiments

#### Generate Problem Instance

In [28]:
def image_to_grid_graph(gray_img, sigma=0.5):
  h, w = gray_img.shape
  # Initialize graph nodes and edges
  nodes = np.zeros((h*w, 1))
  edges = []
  nx_elist = []
  # Compute node potentials and edge weights
  min_weight = 1
  max_weight = 0
  for i in range(h*w):
    x, y = i//w, i%w
    nodes[i] = gray_img[x,y]
    if x > 0:
      j = (x-1)*w + y
      weight = 1-np.exp(-((gray_img[x,y] - gray_img[x-1,y])**2) / (2 * sigma**2))
      edges.append((i, j, weight))
      nx_elist.append(((x,y),(x-1,y),np.round(weight,2)))
      if min_weight>weight:min_weight=weight
      if max_weight<weight:max_weight=weight
    if y > 0:
      j = x*w + y-1
      weight = 1-np.exp(-((gray_img[x,y] - gray_img[x,y-1])**2) / (2 * sigma**2))
      edges.append((i, j, weight))
      nx_elist.append(((x,y),(x,y-1),np.round(weight,2)))
      if min_weight>weight:min_weight=weight
      if max_weight<weight:max_weight=weight
  a=-1
  b=1
  if max_weight-min_weight:
    normalized_edges = [(node1,node2,-1*np.round(((b-a)*((edge_weight-min_weight)/(max_weight-min_weight)))+a,2)) for node1,node2,edge_weight in edges]
    normalized_nx_elist = [(node1,node2,-1*np.round(((b-a)*((edge_weight-min_weight)/(max_weight-min_weight)))+a,2)) for node1,node2,edge_weight in nx_elist]
  else:
    normalized_edges = [(node1,node2,-1*np.round(edge_weight,2)) for node1,node2,edge_weight in edges]
    normalized_nx_elist = [(node1,node2,-1*np.round(edge_weight,2)) for node1,node2,edge_weight in nx_elist]
  return nodes, edges, nx_elist, normalized_edges, normalized_nx_elist

In [30]:
def generate_problem_instance(height,width):
  image = np.random.rand(height,width)
  pixel_values, elist, nx_elist, normalized_elist, normalized_nx_elist = image_to_grid_graph(image)
  G = nx.grid_2d_graph(image.shape[0], image.shape[1])
  G.add_weighted_edges_from(normalized_nx_elist)
  return G, image

In [31]:
def generate_binary_problem_instance(height,width):
  image = np.random.rand(height,width)
  image[image < 0.5] = 0
  image[image >= 0.5] = 1
  pixel_values, elist, nx_elist, normalized_elist, normalized_nx_elist = image_to_grid_graph(image)
  G = nx.grid_2d_graph(image.shape[0], image.shape[1])
  G.add_weighted_edges_from(normalized_nx_elist)
  return G, image

In [35]:
def draw(G, image):
  pixel_values = image
  plt.figure(figsize=(8,8))
  default_axes = plt.axes(frameon=True)
  pos = {(x,y):(y,-x) for x,y in G.nodes()}
  nx.draw_networkx(G,
                  pos=pos,
                  node_color=1-pixel_values,
                  with_labels=True,
                  node_size=3000,
                  cmap=plt.cm.Greys,
                  alpha=0.8,
                  ax=default_axes)
  nodes = nx.draw_networkx_nodes(G, pos, node_color=1-pixel_values,
                  node_size=3000,
                  cmap=plt.cm.Greys)
  nodes.set_edgecolor('k')
  edge_labels = nx.get_edge_attributes(G, "weight")
  nx.draw_networkx_edge_labels(G,
                              pos=pos,
                             edge_labels=edge_labels)

#### Parametric Gate Encoding (PGE)

In [37]:
def R(theta):
  if abs(theta) > 2*np.pi or abs(theta) < 0:
    theta = abs(theta) - (np.floor(abs(theta)/(2*np.pi))*(2*np.pi))
  return 0 if 0 <= theta < np.pi else 1

def cost_fn(params, hermitian_observable):
  global optimization_iteration_count
  optimization_iteration_count += 1
  N = int(np.log2(len(params)))
  circuit_psi = QuantumCircuit(N)
  for i in range(N):
    circuit_psi.h(i)
  diagonal_elements = [np.exp(1j * np.pi * R(param)) for param in params]
  diag_gate = Diagonal(diagonal_elements)
  circuit_psi.append(diag_gate, range(N))
  op_observable = SparsePauliOp.from_operator(hermitian_observable)
  cost = Estimator().run(circuit_psi, op_observable).result().values[0]
  print(f'@ Iteration {optimization_iteration_count} Cost :',cost)
  return cost

def decode(optimal_params):
  return [R(param) for param in optimal_params]


def objective_value(x: np.ndarray, w: np.ndarray) -> float:
    cost = 0
    for i in range(len(x)):
        for j in range(len(x)):
            cost = cost + w[i,j]*x[i]*(1-x[j])
    return cost

In [38]:
def new_nisq_ga_solver2(G, population_size=50, crossover_probability=0.7, mutation_probability=0.2, number_of_generations=50):
    n = G.number_of_nodes()
    w = nx.adjacency_matrix(G).todense()
    D_G = np.diag(list(dict(G.degree()).values()))
    A_G = w
    L_G = D_G - A_G
    n_padding = (2**int(np.ceil(np.log2(n))) - n)
    L_G = np.pad(L_G, [(0, n_padding), (0, n_padding)], mode='constant')
    H = L_G

    # Genetic Algorithm Setup
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_float", random.uniform, 0.5, 2*np.pi)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Evaluate function with padding
    def evaluate_with_padding(individual):
        padded_individual = individual + [0] * n_padding  # Append constant values for padding
        return cost_fn(padded_individual, H),

    toolbox.register("evaluate", evaluate_with_padding)
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # Run Genetic Algorithm
    population = toolbox.population(n=population_size)
    algorithms.eaSimple(population, toolbox, cxpb=crossover_probability, mutpb=mutation_probability, ngen=number_of_generations, verbose=False)

    # Extract the best solution
    best_ind = tools.selBest(population, 1)[0]
    optimal_params = best_ind + [0] * n_padding  # Append constant values for padding
    expectation_value = best_ind.fitness.values[0]

    # Decode and calculate cut value
    x = np.real(decode(optimal_params))
    x = x[:n]
    cut_value = objective_value(x, w)

    return x, expectation_value, cut_value


In [39]:
def new_nisq_algo_solver(G, optimizer_method = 'Genetic', initial_params_seed=123):
  global optimization_iteration_count
  optimization_iteration_count = 0
  if optimizer_method == 'Genetic':
    x, expectation_value, cut_value = new_nisq_ga_solver2(G,
                                                         population_size = 50,
                                                         crossover_probability = 0.7,
                                                         mutation_probability = 0.2,
                                                         number_of_generations = 50)
    success_flag = True
  else:
    n = G.number_of_nodes()
    w = nx.adjacency_matrix(G).todense()
    D_G = np.diag(list(dict(G.degree()).values()))
    A_G = w
    L_G = D_G - A_G
    n_padding = (2**int(np.ceil(np.log2(n)))-n)
    L_G = np.pad(L_G, [(0, n_padding), (0, n_padding) ], mode='constant')
    H = L_G
    np.random.seed(seed=initial_params_seed)
    initial_params = np.random.uniform(low=0.5, high=2*np.pi , size=(n+n_padding))

    options = {}
    result = minimize(
        fun=cost_fn,
        x0=initial_params,
        args=H,
        method=optimizer_method,
        options=options)
    
    optimal_params, expectation_value = result.x, result.fun
    x = np.real(decode(optimal_params))
    x = x[:n]
    cut_value = objective_value(x, w)
    success_flag = result.success
  return success_flag, x, expectation_value, cut_value

#### Experiments for New NISQ

In [56]:
heights = [2,4,8]

scipy_optimizer_methods = ["COBYLA", "Powell", "CG", "BFGS", "L-BFGS-B", "SLSQP", "Genetic"]

initial_params_seed = 123

optimization_iteration_count = 0

base_path = 'results/'

seeds = [111, 222,333,444,555]


In [ ]:

for seed in seeds:
  report_filename = base_path + 'ParameterEncoding_' +  str(seed) + '.txt'
  for height in heights:
    width = height
    print(f'height: {height}, width: {width}, n: {height*width}')
    np.random.seed(seed=seed)
    G, image = generate_problem_instance(height, width)
    print("Image Generated: ",image)
    # plt.imshow(image, cmap=plt.cm.gray)
    # plt.show()
  
    for optimizer_method in scipy_optimizer_methods:
      print(f"Executing QC with {optimizer_method} optimizer for {height}*{height} image.")
      try:
        start_time = time.time()
        success_flag, new_nisq_solution, new_nisq_expectation_value, new_nisq_cut_value = new_nisq_algo_solver(G, 
                                                                                                               optimizer_method = optimizer_method, 
                                                                                                               initial_params_seed=initial_params_seed)
        new_nisq_tte = (time.time() - start_time)
        print(new_nisq_solution, new_nisq_expectation_value, new_nisq_cut_value)
      except:
        print(f"Execution Failed for {optimizer_method} optimizer for {height}*{height} image.")
        continue
      print("New NISQ done for",optimizer_method,"optimizer with a success status :", success_flag)
      print(f"Appending the results of {height}*{height} image using QC with and {optimizer_method} optimizer.")
      row = []
      row.append(int(G.number_of_nodes()))
      row.append(int(height))
      row.append(int(width))
      row.append(success_flag)
      row.append(''.join(map(str, map(int, (new_nisq_solution)))))
      row.append(np.round(new_nisq_tte,6))
      row.append(np.round(new_nisq_cut_value,4))
      row.append(np.round(new_nisq_expectation_value,4))
      row.append(optimization_iteration_count)
      row.append(optimizer_method)
      report_file_obj = open(os.path.join(report_filename),'a+')
      report_file_obj.write('__'.join(map(str,row))+'\n')
      report_file_obj.close()
    print('\n')